# Reconhecimento de imagens baseado em histograma

## Package imports

In [4]:
import os
from skimage import io

## Hyper parâmetros e constantes

In [1]:
path = "dataset/Vistex"

## Entrada

In [3]:
input_img = "c001_001"
n_imgs = 2

## Read images

In [15]:
def sorted_imgs(dir):
    return sorted(os.listdir(dir), key=lambda x: x.split('.')[0])

In [16]:
imgs = [os.path.join(path, x) for x in sorted_imgs(path) ]

for img_name in imgs:
    img = io.imread(img_name)


['dataset/Vistex/c001_001.png',
 'dataset/Vistex/c001_002.png',
 'dataset/Vistex/c001_003.png',
 'dataset/Vistex/c001_004.png',
 'dataset/Vistex/c001_005.png',
 'dataset/Vistex/c001_006.png',
 'dataset/Vistex/c001_007.png',
 'dataset/Vistex/c001_008.png',
 'dataset/Vistex/c001_009.png',
 'dataset/Vistex/c001_010.png',
 'dataset/Vistex/c001_011.png',
 'dataset/Vistex/c001_012.png',
 'dataset/Vistex/c001_013.png',
 'dataset/Vistex/c001_014.png',
 'dataset/Vistex/c001_015.png',
 'dataset/Vistex/c001_016.png',
 'dataset/Vistex/c002_001.png',
 'dataset/Vistex/c002_002.png',
 'dataset/Vistex/c002_003.png',
 'dataset/Vistex/c002_004.png',
 'dataset/Vistex/c002_005.png',
 'dataset/Vistex/c002_006.png',
 'dataset/Vistex/c002_007.png',
 'dataset/Vistex/c002_008.png',
 'dataset/Vistex/c002_009.png',
 'dataset/Vistex/c002_010.png',
 'dataset/Vistex/c002_011.png',
 'dataset/Vistex/c002_012.png',
 'dataset/Vistex/c002_013.png',
 'dataset/Vistex/c002_014.png',
 'dataset/Vistex/c002_015.png',
 'datase

## PDF and histogram functions

In [ ]:
def hist(src_img):
    

def dist_pdf(src_img, comp_img):
    pass